In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from skimage import filters
from skimage import morphology
import math
from skimage import measure
import time
import sklearn.preprocessing
from sklearn.ensemble import RandomForestClassifier

In [3]:
def img_normalize(sv):
    mx, mn=sv.max(), sv.min()
    sv=(sv-mn)/(mx-mn)
    sv=np.asarray(sv*255, dtype='uint8')
    return sv


dt=[('0' if x<10 else '')+str(x) for x in range(1, 15)]
dy=[[y+'_'+x for y in dt] for x in ['dr', 'h', 'g']]
z=[]
for x in dy:
    z=z+x
print(z)

flag=cv2.IMREAD_GRAYSCALE
#flag=cv2.IMREAD_COLOR
Manuals = [cv2.imread(f"manual1/{x}.tif", flag) for x in z]
Masks= [cv2.imread(f"mask/{x}_mask.tif", flag) for x in z]
Images= [cv2.imread(f"images/{x}.jpg", flag) for x in z]

zupaImagez=[cv2.imread(f"images/{x}.jpg", cv2.IMREAD_COLOR) for x in z]

Creation=list(zip(Manuals, Masks, Images, zupaImagez))

['01_dr', '02_dr', '03_dr', '04_dr', '05_dr', '06_dr', '07_dr', '08_dr', '09_dr', '10_dr', '11_dr', '12_dr', '13_dr', '14_dr', '01_h', '02_h', '03_h', '04_h', '05_h', '06_h', '07_h', '08_h', '09_h', '10_h', '11_h', '12_h', '13_h', '14_h', '01_g', '02_g', '03_g', '04_g', '05_g', '06_g', '07_g', '08_g', '09_g', '10_g', '11_g', '12_g', '13_g', '14_g']


In [33]:
def pad(img, con=3):
    if (len(img.shape)==2):
        dt=np.zeros((img.shape[0]+2*con, img.shape[1]+2*con), dtype='uint8')
        dt[con:dt.shape[0]-con,con:dt.shape[1]-con]=img
    else:
        dt=np.zeros((img.shape[0]+2*con, img.shape[1]+2*con, img.shape[2]), dtype='uint8')
        dt[con:dt.shape[0]-con,con:dt.shape[1]-con]=img
    
    return dt

def getItem(img, x, y, con=3, img2=None):
    #huhu=measure.moments_hu(img[x-con:x+con,y-con:y+con])
    if (img2 is None):
        mcen=measure.moments_central(img[x-con:x+con,y-con:y+con])
        vz=[img[x-con:x+con,y-con:y+con].var()]
    else:
        mcen=measure.moments_central(img2[x-con:x+con,y-con:y+con])
        vz=[img2[x-con:x+con,y-con:y+con,z].var() for z in range(3)]
    col=[img[x,y]]
    lees=np.concatenate((vz, col, np.reshape(mcen, mcen.shape[0]*mcen.shape[1]*mcen.shape[2])))
    return lees

def getImagoData(img, imgalc, mask):
    Cnt=5
    #dissolve=np.zeros((img.shape[0]*img.shape[1], getItem(img, 200, 200).shape[0]))
    dissolve=[]
    img2=pad(img, Cnt)
    img3=pad(imgalc, Cnt)
    
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            #dissolve[i*img.shape[0]+j]=getItem(img2, i+Cnt, j+Cnt, Cnt, img3)
            if (mask[i,j]!=0):
                dissolve.append(getItem(img2, i+Cnt, j+Cnt, Cnt, img3))
    return dissolve


def getImagoRes(bwimg):
    res=np.reshape(bwimg, (bwimg.shape[0]*bwimg.shape[1]))
    return res

def getImagoRandomoRes(resimg, tupls):
    res=[]
    for x in tupls:
        res.append(resimg[x[0], x[1]])
    return np.asarray(res)

#0 - manual, 1 - maska, 2 - oryginał
def randomizer(set_img, Z=40001):
    Cnt=5
    img2=pad(set_img[2], Cnt)
    img3=pad(set_img[3], Cnt)
    dissolve=[]
    tuplez=[]
    setall=set()
    
    for i in range(Z):
        while (True):
            rn=np.random.randint(0, set_img[0].shape[0]*set_img[0].shape[1])
            rny=rn%set_img[1].shape[1]
            rnx=rn//set_img[1].shape[1]
                
            if (set_img[1][rnx,rny]!=0 and
                (set_img[0][rnx,rny]!=0 or
                 np.random.randint(0, 4)%4==0) and
               {(rnx, rny)} not in setall):
                break
        dissolve.append(getItem(img2, rnx+Cnt, rny+Cnt, Cnt, img3))
        setall|={(rnx, rny)}
        tuplez.append((rnx, rny))
        #print(rnx, rny, set_img[0][rnx,rny])
    return np.asarray(dissolve), tuplez

def preproc(data):
    dv=sklearn.preprocessing.normalize(data)
    return dv


def evaluator(im1, im2):
    dicc={}
    dicc["1st B"]=len(im1[im1==0])
    dicc["1st W"]=len(im1[im1!=0])
    dicc["2nd B"]=len(im2[im2==0])
    dicc["2nd W"]=len(im2[im2!=0])
    
    dicc["tp"]=len(im2[(im2==im1) & (im2!=0)])
    dicc["tn"]=len(im2[(im2==im1) & (im2==0)])
    dicc["fp"]=len(im2[(im2!=im1) & (im2!=0)])
    dicc["fn"]=len(im2[(im2!=im1) & (im2==0)])
    _=[print(x, dicc[x]) for x in ["tp", "tn", "fp", "fn"]]
    if (dicc["fn"]==0):
        dicc["fn"]=1
    
    dicc["acc"]=(dicc["tp"]+dicc["tn"])/(dicc["tp"]+dicc["tn"]+dicc["fp"]+dicc["fn"])
    dicc["prec"]=dicc["tp"]/(dicc["tp"]+dicc["fp"])
    dicc["rec"]=dicc["tp"]/(dicc["tp"]+dicc["fn"])
    
    return dicc

In [8]:
x1=[]
y1=[]

for i in [0, 7, 11]:
    xv, locsv=randomizer(Creation[i])
    yv=getImagoRandomoRes(Creation[i][0], locsv)
    x1=x1+list(xv)
    y1=y1+list(yv)
    print(i)
x1=np.asarray(x1)
y1=np.asarray(y1)
    
##Normalizacja trainseta, uwypuklenie kluczowych cech
xp1=preproc(x1)

0
7
11
87.42076539993286


In [32]:
KValid=4

def f(x, l, val):
    s=l//val
    return x//s

Gr=[f(x, len(x1)+1, KValid) for x in range(len(x1))]
np.random.shuffle(Gr)
Gr=np.asarray(Gr)

30000


In [38]:
def validato(ter, Gr, KV, x, y):
    summa=0
    for numb in range(KV):
        rfc1=RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=ter)
        rfc1.fit(x[Gr!=numb], y[Gr!=numb])
        ypt=rfc1.predict(x[Gr==numb])
        dct=evaluator(y[Gr==numb], ypt)
        summa+=math.sqrt(dct['prec']*dct['rec'])
    return summa/KV

l, r=0, len(x1)
while (l<r):
    a=(r-l)//3
    b=2*(r-l)//3
    
    curres1=validato(a, Gr, KValid, xp1, y1)
    curres2=validato(b, Gr, KValid, xp1, y1)
    if (curres1<curres2):
        l=a
    elif (curres1>curres2):
        r=b
    else:
        l, r=a, b
    
    print(l, r, curres1, curres2)


rfc=RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=a)
rfc.fit(xp1, y1)

tp 5003
tn 21417
fp 1115
fn 2466
tp 5000
tn 21498
fp 1120
fn 2383
tp 5116
tn 21425
fp 1071
fn 2389
tp 5065
tn 21292
fp 1089
fn 2554
tp 5052
tn 21449
fp 1083
fn 2417
tp 5045
tn 21514
fp 1104
fn 2338
tp 5123
tn 21398
fp 1098
fn 2382
tp 5044
tn 21303
fp 1078
fn 2575
40001 120003 0.7436054364750464 0.745845677470991
tp 5033
tn 21433
fp 1099
fn 2436
tp 5019
tn 21506
fp 1112
fn 2364
tp 5106
tn 21378
fp 1118
fn 2399
tp 5070
tn 21305
fp 1076
fn 2549
tp 5060
tn 21454
fp 1078
fn 2409
tp 5037
tn 21535
fp 1083
fn 2346
tp 5120
tn 21373
fp 1123
fn 2385
tp 5087
tn 21286
fp 1095
fn 2532
26667 120003 0.7444199036577012 0.7464689624355417
tp 5050
tn 21433
fp 1099
fn 2419
tp 5005
tn 21533
fp 1085
fn 2378
tp 5135
tn 21364
fp 1132
fn 2370
tp 5084
tn 21311
fp 1070
fn 2535
tp 4989
tn 21429
fp 1103
fn 2480
tp 5000
tn 21525
fp 1093
fn 2383
tp 5096
tn 21367
fp 1129
fn 2409
tp 5061
tn 21296
fp 1085
fn 2558
26667 62224 0.7457006032730141 0.7425614487363824
tp 5044
tn 21467
fp 1065
fn 2425
tp 5026
tn 21516
fp 1102

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2795, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [9]:
ln, sgeom=20, 0
for i in range(0, ln):
    t1=time.time()
    
    xt, locst=randomizer(Creation[i])
    yt=getImagoRandomoRes(Creation[i][0], locst)
    xpt=preproc(xt)
    ypt=rfc.predict(xpt)
    dct=evaluator(yt, ypt)
    print(f"Obraz {i+1}:")
    print(f"acc:{dct['acc']}, prec:{dct['prec']}, rec:{dct['rec']}, geom:{math.sqrt(dct['prec']*dct['rec'])}")
    
    t2=time.time()
    print(t2-t1)
    sgeom=sgeom+math.sqrt(dct['prec']*dct['rec'])
print(f"result: {sgeom/ln}")

tp 6094
tn 30110
fp 1808
fn 1989
Obraz 1:
acc:0.9050773730656734, prec:0.771197165274614, rec:0.7539279970308054, geom:0.7625136943893694
8.569417476654053
tp 6617
tn 27922
fp 2266
fn 3196
Obraz 2:
acc:0.8634534136646583, prec:0.7449060002251492, rec:0.6743095893202894, geom:0.708729327101708
8.555994987487793
tp 7298
tn 27986
fp 2459
fn 2258
Obraz 3:
acc:0.8820779480512987, prec:0.7479758122373681, rec:0.7637086647132691, geom:0.7558013024609201
8.523391723632812
tp 5156
tn 28618
fp 2560
fn 3667
Obraz 4:
acc:0.8443288917777055, prec:0.6682218766200103, rec:0.5843817295704409, geom:0.6248973163616622
8.641691207885742
tp 9416
tn 7123
fp 23140
fn 322
Obraz 5:
acc:0.4134646633834154, prec:0.28922472048163167, rec:0.9669336619429041, geom:0.5288299519691722
8.467015981674194
tp 10692
tn 8748
fp 19970
fn 591
Obraz 6:
acc:0.4859878503037424, prec:0.3487052377535712, rec:0.947620313746344, geom:0.5748392530134252
8.502798557281494
tp 8310
tn 25565
fp 2995
fn 3131
Obraz 7:
acc:0.8468538286542